
# Continuous Control

---

This notebook guides you through the creation of an agent that will learn to pass the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

## 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

## 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
skipRandom = True

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


## 3. Hyperparameter and Architecture Search

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import matplotlib.pyplot as plt


### 3.1 Model Training Function

In [7]:
from ddpg_agent import *
def train_ddpg(batch_size, buffer_size, use_batch_norm, random_fn, num_episodes):
    agent = Agent(numAgents = num_agents, 
                  state_size=state_size, 
                  action_size=action_size, 
                  random_seed=47,
                  batch_size = batch_size,
                  buffer_size = buffer_size,
                  use_batch_norm = use_batch_norm,
                  random_fn = random_fn)
    
    
    scores = []
    PRINT_EVERY=1
    for episode in range(num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        agent.reset()

        score = np.zeros(num_agents)
        while True:
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            agent.step(states, actions, rewards, next_states, dones)
            score += np.array(rewards)
            states = next_states
            if np.any(dones):
                break
        if episode % PRINT_EVERY == 0:
            print("Episode {} Min/Avg/Max {:.4E}/{:.4E}/{:.4E}".format(
                episode, np.min(score), np.mean(score), np.max(score)))
        scores.append(np.mean(score))
    return scores, agent

### 3.2 Hyperparameter and Model Architecture Search
Here, we evaluate the performance over 50 episodes of three different batch sizes [64,128,256], two different buffer sizes [1e5,1e6], and two different network architectures (with and without a batch normalization layer).

In [8]:
import random
import time
import pickle
import torch
import numpy as np

skipSearch = True
if not skipSearch:
    for batch_size in [64,128,256]:
        for buffer_size in [100000,1000000]:
            for use_batch_norm in [False,True]:
                for random_fn, fn_name in [(np.random.randn, 'normal')]:
                    start_time = time.time()
                    scores, agent = train_ddpg(batch_size,
                                       buffer_size,
                                       use_batch_norm,
                                       random_fn,
                                       50)
                    end_time = time.time()
                    fname_suffix = '{}_{}_{}_{}'.format(batch_size, buffer_size, use_batch_norm, fn_name)
                    fname = './data/time_and_scores_{}.pkl'.format(fname_suffix)
                    pickle.dump((end_time-start_time, scores), open(fname, 'wb'))
                    torch.save(agent.actor_local.state_dict(), './data/actor_{}.pt'.format(fname_suffix))
                    torch.save(agent.critic_local.state_dict(), './data/critic_{}.pt'.format(fname_suffix))
                
                
                

### 3.3 Train the Full Model
Train a model over 300 episodes using the mini-batch size of 128, the buffer size of 1e6, and using the batch normalization layer

In [9]:
skipModelTraining = True
if not skipModelTraining:
    start_time = time.time()
    batch_size = 128
    buffer_size = 1000000
    use_batch_norm = True
    fn_name = 'normal'
    fname_suffix = '{}_{}_{}_{}'.format(batch_size, buffer_size, use_batch_norm, fn_name)

    scores, agent = train_ddpg(batch_size,buffer_size,use_batch_norm,np.random.randn, 300)
    end_time = time.time()
    fname = './data/time_and_scores_{}_official.pkl'.format(fname_suffix)

    pickle.dump((end_time-start_time, scores), open(fname, 'wb'))
    torch.save(agent.actor_local.state_dict(), './data/actor_{}_official.pt'.format(fname_suffix))
    torch.save(agent.critic_local.state_dict(), './data/critic_{}_official.pt'.format(fname_suffix))

## 4. Pre-Trained Model Results

In [10]:
# Load saved models into memory
from ddpg_agent import Agent
import torch
import numpy as np
from unityagents import UnityEnvironment
import numpy as np
import pickle


skipPreTrained = True
if not skipPreTrained:
    # close an instance if it exists
    try: env
    except NameError: env = None
    if env:
        env.close()
    env = UnityEnvironment(file_name='./Reacher_Linux/Reacher.x86_64')
    # get the default brain
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    env_info = env.reset(train_mode=True)[brain_name]

    # number of agents
    num_agents = len(env_info.agents)
    print('Number of agents:', num_agents)

    # size of each action
    action_size = brain.vector_action_space_size
    print('Size of each action:', action_size)

    # examine the state space 
    states = env_info.vector_observations
    state_size = states.shape[1]
    numAgents = 20
    agent = Agent(numAgents = numAgents, 
                      state_size=state_size, 
                      action_size=action_size, 
                      random_seed=47,
                      use_batch_norm = True,
                      random_fn = np.random.randn)
    agent.actor_local.load_state_dict(torch.load("./data/actor_128_1000000_True_normal_official.pt"))

    numEpisodes = 5

    mean_scores = []
    for i in range(numEpisodes):
        env_info = env.reset(train_mode=False)[brain_name] # reset the environment
        states = env_info.vector_observations            # get the current state
        scores = np.zeros(numAgents)
        while True:
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            scores += env_info.rewards
            states = next_states
        
            if np.any(dones):
                break
        print("{}) Average Score: {}".format(i, np.mean(scores)))
        mean_scores.append(np.mean(scores))
    fname = './data/trained_model_scores.pkl'
    pickle.dump(mean_scores, open(fname, 'wb'))

